In [1]:
# 安装必要的库
!pip install pandas
!pip install requests

# 导入库
import pandas as pd
import requests

# 加载CSV文件
csv_url = "https://aistdancedb.ongaaccel.jp/data/video_refined/10M/refined_10M_sFM_all.csv"
df = pd.read_csv(csv_url)

# 根据CAMERA=c01筛选数据
filtered_df = df[df['CAMERA'] == 'c01']

# 打印筛选后的数据以检查
print(filtered_df.head())
# 检查一共筛选出多少数据
print(f"Total number of filtered videos: {len(filtered_df)}")


Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
                                                 URL  \
0  https://aistdancedb.ongaaccel.jp/v1.0.0/video/...   
1  https://aistdancedb.ongaaccel.jp/v1.0.0/video/...   
2  https://aistdancedb.ongaaccel.jp/v1.0.0/video/...   
3  https://aistdancedb.ongaaccel.jp/v1.0.0/video/...   
4  https://aistdancedb.ongaaccel.jp/v1.0.0/video/...   

                       FILE_NAME                  BASE_NAME        SEC  \
0  gBR_sFM_c01_d04_mBR0_ch01.mp4  gBR_sFM_c01_d04_mBR0_ch01  47.964583   
1  gBR_sFM_c01_d04_mBR1_ch02.mp4  gBR_sFM_c01_d04_mBR1_ch02  42.625917   
2  gBR_sFM_c01_d04_mBR2_ch03.mp4  gBR_sFM_c01_d04_mBR2_ch03  38.371667   
3  gBR_sFM_c01_d04_mBR3_ch04.mp4  gBR_sFM_c01_d04_mBR3_ch04  34.884850   
4  gBR_sFM_c01_d04_mBR4_ch05.mp4  gBR_sFM_c01_d04_mBR4_ch05  31.981950   

   FILE_SIZE GENRE SITUATION CAMERA DANCER MUSIC CHOREOGRAPHY  
0   62212252   gBR       sFM    c0

In [2]:
import os
# 创建目录（如果不存在）
os.makedirs("DanceGenrevideos", exist_ok=True)

# 下载视频
def download_video(url, filename):
    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)

# 遍历筛选后的URL并下载视频
for index, row in filtered_df.iterrows():
    video_url = row['URL']
    filename = "DanceGenrevideos/" +row['FILE_NAME']
    download_video(video_url, filename)
    print(f"Downloaded {filename}")

print("All videos downloaded.")
# 检查当前目录
print("Current directory:")
!pwd

# 检查下载到的文件位置
print("Files downloaded to:")
!ls DanceGenrevideos


Downloaded DanceGenrevideos/gBR_sFM_c01_d04_mBR0_ch01.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d04_mBR1_ch02.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d04_mBR2_ch03.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d04_mBR3_ch04.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d04_mBR4_ch05.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d04_mBR4_ch07.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d04_mBR5_ch06.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d05_mBR1_ch08.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d05_mBR2_ch09.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d05_mBR3_ch10.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d05_mBR4_ch11.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d05_mBR4_ch13.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d05_mBR5_ch12.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d05_mBR5_ch14.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d06_mBR1_ch15.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d06_mBR2_ch16.mp4
Downloaded DanceGenrevideos/gBR_sFM_c01_d06_mBR3_ch17.mp4
Downloaded Dan

'pwd' 不是内部或外部命令，也不是可运行的程序
或批处理文件。
'ls' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


In [6]:
! git clone https://github.com/CMU-Perceptual-Computing-Lab/openpose.git


Cloning into 'openpose'...
fatal: unable to access 'https://github.com/CMU-Perceptual-Computing-Lab/openpose.git/': Failed to connect to github.com port 443 after 21128 ms: Couldn't connect to server


In [ ]:
import cv2
import numpy as np
import pandas as pd
from openpose import pyopenpose as op

# 配置 OpenPose
params = {"model_folder": "/content/openpose/models"}
opWrapper = op.WrapperPython()
opWrapper.configure(params)
opWrapper.start()

# 打开视频文件
video_path = "path_to_video.mp4"  # 请将此路径替换为你的视频路径
cap = cv2.VideoCapture(video_path)

# 理论上的帧率
frame_rate = 60
num_frames = int(min(10 * frame_rate, cap.get(cv2.CAP_PROP_FRAME_COUNT)))

# 用于保存特征向量的列表
features = []

for frame_number in range(num_frames):
    ret, frame = cap.read()
    if not ret:
        break

    datum = op.Datum()
    datum.cvInputData = frame
    opWrapper.emplaceAndPop([datum])

    if datum.poseKeypoints is not None:
        skeleton = datum.poseKeypoints[0]  # 获取第一个人的骨骼数据
        joint_angles = []

        for joint in skeleton:
            x, y, confidence = joint
            joint_angles.append(np.arctan2(y, x))

        feature_vector = []
        for angle in joint_angles:
            feature_vector.append(np.sin(angle))
            feature_vector.append(np.cos(angle))

        features.append(feature_vector)

# 关闭视频文件
cap.release()

# 保存到CSV文件
df = pd.DataFrame(features, columns=[f"feature_{i+1}" for i in range(42)])
df.to_csv("output_features.csv", index=False)
